In [1]:
import geopandas as gpd
import shapely 
import cartopy.crs as ccrs
import cartopy.feature as feature
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
import matplotlib.pyplot as plt
import matplotlib
import networkx as nx
import folium
import rtree
from tqdm import tqdm
from multiprocessing import Pool
processpool = Pool(processes=7)
from IPython.display import HTML
def iframe(url):
    iframe = '<iframe src=' + url + ' width=700 height=350></iframe>'
    return(HTML(iframe))
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%config Completer.use_jedi = False

In [2]:
parks = gpd.GeoDataFrame.from_file("nps_boundary_shp/nps_boundary.shp").to_crs({'init': 'epsg:3857'})
roads = gpd.GeoDataFrame.from_file("tl_2016_us_primaryroads/tl_2016_us_primaryroads.shp").to_crs({'init': 'epsg:3857'})
cntys = gpd.GeoDataFrame.from_file("/Users/tony/Downloads/tl_2016_us_county/tl_2016_us_county.shp").to_crs({'init': 'epsg:3857'})
wilde = gpd.GeoDataFrame.from_file("S_USA.Wilderness/S_USA.Wilderness.shp").to_crs({'init': 'epsg:3857'})
fores = gpd.GeoDataFrame.from_file("S_USA.AdministrativeForest/S_USA.AdministrativeForest.dbf").to_crs({'init': 'epsg:3857'})


OSError: no such file or directory: '/Users/tony/Downloads/tl_2016_us_county/tl_2016_us_county.shp'

In [3]:
print(len(wilde))
wilde.head()


447


,AREAID,BOUNDARYST,GIS_ACRES,SHAPE_AREA,SHAPE_LEN,WID,WILDERNESS,WILDERNE_1,geometry
0,255,"Provisional, Subject to Change",16008.938704,0.006352,0.569281,107,10661010343,Cedar Bench Wilderness,POLYGON ((-12453602.76816968 4091916.015028701...
1,124,"Provisional, Subject to Change",46052.580517,0.018707,2.170713,109,10546010343,Chama River Canyon Wilderness,(POLYGON ((-11883362.66978141 4354843.99120224...
2,274,"Provisional, Subject to Change",8061.539060,0.003464,0.376443,111,10676010343,Chanchelulla Wilderness,POLYGON ((-13688895.94041893 4933527.994584521...
3,182,Final,12537.514736,0.005278,0.759203,112,10586010343,Charles C. Deam Wilderness,(POLYGON ((-9618856.641707031 4725799.08368899...
4,184,Final,7399.780494,0.002903,0.349242,115,10588010343,Cheaha Wilderness,POLYGON ((-9550964.422707578 3955843.284075238...


In [4]:
print(len(fores))
fores.head()

112


,ADMINFORES,FORESTNAME,FORESTNUMB,FORESTORGC,GIS_ACRES,REGION,SHAPE_AREA,SHAPE_LEN,geometry
0,99011400010343,Kootenai National Forest,14,0114,2624578.394,01,1.290843,21.874705,(POLYGON ((-12829337.62425734 6142169.69229397...
1,99030400010343,Coconino National Forest,04,0304,2000935.403,03,0.798729,7.081378,"POLYGON ((-12441690.4004418 4243713.646472579,..."
2,99020600010343,Medicine Bow-Routt National Forest,06,0206,4592727.645,02,2.024169,28.041577,(POLYGON ((-11764248.48137607 5345369.12535563...
3,99031200010343,Tonto National Forest,12,0312,2966417.901,03,1.169763,7.912869,(POLYGON ((-12384093.16045861 4089570.31855299...
4,99030200010343,Carson National Forest,02,0302,1593021.419,03,0.648673,11.525884,(POLYGON ((-11816179.94110374 4432908.19422580...


In [5]:
print(len(parks))
parks.head()

510


,DATE_EDIT,GIS_LOC_ID,GIS_NOTES,GROUP_CODE,LANDS_CODE,META_MIDF,UNIT_CODE,UNIT_NAME,UNIT_TYPE,geometry,id,observed
0,,,,,,,NACE,Marion Park,Park,POLYGON ((-8571399.866308754 4704907.728417463...,1602740,
1,2006/01/04,,Lands - http://landsnet.nps.gov/tractsnet/docu...,,,,APCO,Appomattox Court House,National Historical Park,(POLYGON ((-8771068.516886771 4494046.73973837...,1604107,
2,2007/07/10,,Lands - http://landsnet.nps.gov/tractsnet/docu...,,,,ORPI,Organ Pipe Cactus,National Monument,"POLYGON ((-12588495.71991122 3758224.23773837,...",1604358,
3,,,Shifted 0.06 miles,,,,PINN,Pinnacles,National Monument,(POLYGON ((-13496337.57336841 4376047.48085925...,1604365,
4,,,Good,,,,TUIN,Tuskegee Institute,National Historical Site,(POLYGON ((-9540237.975449599 3819398.07320702...,1604368,


In [6]:
parks['AREA'] = parks.geometry.apply(lambda x: x.area)
print(parks.AREA.max(), parks.AREA.mean(), parks.AREA.std())
#parks = parks[parks.AREA < parks.AREA.max() - 1e9]
parks = parks[parks.UNIT_TYPE == "National Park"]

204263715208.0 2679242793.67 14812001433.6


In [7]:
color_forests = '#a6cee3'
color_parks = '#1f78b4'
color_wilderness = '#b2df8a'

---


In [10]:
def min_distance(point, polys):
    return polys.distance(point).min()

In [11]:
def parkdist(x):
    return(min_distance(x, parks))
def forestdist(x):
    return(min_distance(x, fores))
def wilderdist(x):
    return(min_distance(x, wilde))

cntys['min_park_dist'] = list(processpool.map(parkdist, cntys.geometry.centroid))


In [12]:
cntys['min_forest_dist'] =  list(processpool.map(forestdist, cntys.geometry.centroid))


In [13]:
cntys['min_wilder_dist'] = list(processpool.map(wilderdist, cntys.geometry.centroid))

In [14]:
extent = [-130, -65, 24, 50]
## Set Up Plot
fig, ax = plt.subplots(3, 1, figsize=(20, 20), subplot_kw={'projection': ccrs.Mercator().GOOGLE})

cmap = plt.cm.Blues
norm = matplotlib.colors.Normalize(vmin = 0, vmax = 1000000)



for axidx, dist_type in enumerate(['min_park_dist', 'min_forest_dist', 'min_wilder_dist']):
    ax[axidx].set_extent(extent, ccrs.Geodetic()) # North America, in Geodetic CRS
    #ax[axidx].set_title("Distance from National Parklands", loc='left', fontsize=22)

    ## Add Gridlines
    gl = ax[axidx].gridlines(crs=ccrs.PlateCarree(), draw_labels =True,
                      linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
    gl.xlabels_top = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER

    ## Add title
    if (dist_type == 'min_park_dist'):
        title = "National Parks"
    elif dist_type == 'min_forest_dist':
        title = "National Forest Lands"
    elif dist_type == 'min_wilder_dist':
        title = "Wilderness Areas"
    ax[axidx].set_title(title, fontsize=15, loc='left', weight='light')
    
    ## Add Counties
    for _, county in cntys.iterrows():
        ax[axidx].add_geometries([county['geometry']],
                          crs=ccrs.Mercator().GOOGLE, 
                          facecolor=cmap(norm(county[dist_type])), 
                          edgecolor='none')



    ## Add States
    ax[axidx].add_feature(feature.NaturalEarthFeature(category='cultural',
                                                 name='admin_1_states_provinces_lakes_shp', 
                                                 scale='50m',
                                                 facecolor='none', 
                                                edgecolor='black', 
                                              linewidth=0.8))
    ax[axidx].add_feature(feature.BORDERS)

    ## Add National Parks
    for _, park in parks.iterrows():
        ax[axidx].add_geometries([park['geometry']], crs=ccrs.Mercator().GOOGLE, facecolor=color_parks, edgecolor='none')




    ## Add Forests
    ax[axidx].add_geometries(fores.geometry, crs=ccrs.Mercator().GOOGLE, facecolor=color_forests, edgecolor='none')

    ## Add Wildernesses
    ax[axidx].add_geometries(wilde.geometry, crs=ccrs.Mercator().GOOGLE, facecolor=color_wilderness, edgecolor='none')

    ## Add Legend
    forest_patch = matplotlib.patches.Patch(color=color_forests, label = "National Forests")
    park_patch = matplotlib.patches.Patch(color=color_parks, label = 'National Parks')
    wilder_patch = matplotlib.patches.Patch(color=color_wilderness, label = "Wilderness Areas")
    ax[axidx].legend(handles=[forest_patch, 
                       park_patch,
                       wilder_patch], labels=['National Forests','National Parks','Wilderness Areas'], 
                    fontsize=10, 
                    loc='lower left')

    ## Add Roads
    ax[axidx].add_geometries(roads[(roads.RTTYP == 'I') | (roads.RTTYP == 'M')].geometry, crs=ccrs.Mercator().GOOGLE, facecolor='none', edgecolor='red', linewidth=0.2)

sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cb = fig.colorbar(sm, orientation='vertical', ax=ax.ravel().tolist(), aspect=50)
cb.set_label("Distance (m)")

fig.savefig('westconus_distance.png', bbox_inches='tight', transparent=True, dpi = 1200)
fig.show()

/Users/tony/miniconda3/lib/python3.5/site-packages/matplotlib/figure.py:397: UserWarning: matplotlib is currently using a non-GUI backend, so cannot show the figure
  "matplotlib is currently using a non-GUI backend, "
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.
